This notebook was created to support the data preparation required to support our CS 598 DLH project.  The paper we have chosen for the reproducibility project is:
***Ensembling Classical Machine Learning and Deep Learning Approaches for Morbidity Identification from Clinical Notes ***



 

The data cannot be shared publicly due to the agreements required to obtain the data so we are storing the data locally and not putting in GitHub.

In [366]:
import os
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import datetime
from datetime import timedelta
from tqdm import tqdm
import torchtext
from torch.utils.data import SubsetRandomSampler
from sklearn.model_selection import KFold,train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import tensorflow_hub as hub
from nltk.tokenize import sent_tokenize

# set seed
seed = 24
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
# define data path
DATA_PATH = './obesity_data/'
RESULTS_PATH = './results/'
MODELS_PATH = './models/'

if os.path.exists(RESULTS_PATH) == False:
    os.mkdir(RESULTS_PATH)
if os.path.exists(MODELS_PATH) == False:
    os.mkdir(MODELS_PATH)


#test_df = pd.read_pickle(DATA_PATH + '/test.pkl') 
#train_df = pd.read_pickle(DATA_PATH + '/train.pkl') 
#corpus = pd.read_pickle(DATA_PATH + '/corpus.pkl')
all_df = pd.read_pickle(DATA_PATH + '/all_df.pkl') 
all_df_expanded = pd.read_pickle(DATA_PATH + '/all_df_expanded.pkl')

#Get this from the create embeddings file
max_tokens = 1416
max_sentences = 380

#Download info for USE
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")


disease_list = all_df['disease'].unique().tolist()
embedding_list = ['GloVe', "FastText"]
result_cols = ['Batch','Disease','Embedding','AUROC','F1','F1_MACRO', 'F1_MICRO', 'Exec Time', 'Total Run (secs)','Epochs', 'Dropout', 'BatchSize', 'Hidden', 'LR', 'CV']
result_loss_cols = ['Loss','Epoch','Batch','Disease','Embedding','Epochs', 'Dropout', 'BatchSize', 'Hidden', 'LR']

***Common training and evaluation code***

In [367]:
eps=1e-10

def train_model(tmodel, train_dataloader, n_epoch=5, lr=0.003, device=None, model_name='unk', use_decay=False):
    import torch.optim as optim
    
    device = device or torch.device('cpu')
    tmodel.train()

    loss_history = []

    # your code here
    optimizer = optim.Adam(tmodel.parameters(), lr=lr)
    # want to decay the learning rate as teh number of epochs get larger
    #scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma = 0.1)
    if use_decay:
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min',
            factor=0.1, patience=10, threshold=0.0001, threshold_mode='abs')

    #loss_func = nn.BCELoss()
    loss_func = nn.CrossEntropyLoss()
    #loss_func = nn.NLLLoss()

    for epoch in range(n_epoch):
        epoch = epoch+1
        curr_epoch_loss = []
        start = time.time()

        bs = train_dataloader.batch_size
        hs = tmodel.hidden_size
        do = tmodel.dropout

        for X, Y in tqdm(train_dataloader,desc=f"Training {model_name}-Lr{str(lr)}-Epoch{epoch}of{n_epoch}-BatchSize{bs}-HiddenState{hs}-Dropout{do}..."):
            # your code here
            optimizer.zero_grad()

            y_hat = tmodel(X)

            loss = loss_func(y_hat, Y)
            #loss = loss_func(torch.log(y_hat+ eps), Y)
            
            loss.backward()
            optimizer.step()
            if use_decay:
                scheduler.step(loss)
            
            curr_epoch_loss.append(loss.cpu().data.numpy())


        end = time.time()
        if epoch % 10 == 0:
            print(f"epoch{epoch}: curr_epoch_loss={np.mean(curr_epoch_loss)},execution_time={str(datetime.timedelta(seconds = (end-start)))},lr={optimizer.param_groups[0]['lr']}")

        #scheduler.step()
        loss_history += curr_epoch_loss
    return tmodel, loss_history

def eval_model(emodel, dataloader, device=None, model_name='unk'):
    """
    :return:
        pred_all: prediction of model on the dataloder.
        Y_test: truth labels. Should be an numpy array of ints
    TODO:
        evaluate the model using on the data in the dataloder.
        Add all the prediction and truth to the corresponding list
        Convert pred_all and Y_test to numpy arrays 
    """
    device = device or torch.device('cpu')
    emodel.eval()
    pred_all = []
    Y_test = []
    for X, Y in tqdm(dataloader, desc=f"Evaluating {model_name}..."):
        # your code here
        y_hat = emodel(X)
        
        pred_all.append(y_hat.detach().to('cpu'))
        Y_test.append(Y.detach().to('cpu'))
        
    pred_all = np.concatenate(pred_all, axis=0)
    Y_test = np.concatenate(Y_test, axis=0)

    return pred_all, Y_test

In [368]:
def evaluate_predictions(truth, pred):
    """
    TODO: Evaluate the performance of the predictoin via AUROC, and F1 score
    each prediction in pred is a vector representing [p_0, p_1].
    When defining the scores we are interesed in detecting class 1 only
    (Hint: use roc_auc_score and f1_score from sklearn.metrics, be sure to read their documentation)
    return: auroc, f1
    """
    from sklearn.metrics import roc_auc_score, f1_score

    # your code here
    auroc = roc_auc_score(truth, pred[:,1])
    f1 = f1_score(truth, np.argmax(pred,axis=1))
    f1_macro = f1_score(truth, np.argmax(pred,axis=1),average='macro')
    f1_micro = f1_score(truth, np.argmax(pred,axis=1),average='micro')

    return auroc, f1, f1_macro, f1_micro

In [369]:
def trainAndEvaluate(train_loader, val_loader, model, model_desc, batch_name, results_file, disease, lr,  dataformat, embedding, device, n_epoch, do, batch_size, hs, cv, use_decay):
            
    return_val = False

    start_train = time.time()
    model, loss_history = train_model(model, train_loader, n_epoch=n_epoch, lr = lr, device=device, model_name=model_desc, use_decay=use_decay)
    end_train = time.time()

    try:
        #Evaluate model
        start_eval = time.time()
        pred, truth = eval_model(model, val_loader, device=device, model_name=model_desc)
        end_eval = time.time()

        auroc, f1, f1_macro, f1_micro = evaluate_predictions(truth, pred)
        runtime = f"Trn,Eval,Ttl={str(datetime.timedelta(seconds = (end_train-start_train)))},{str(datetime.timedelta(seconds = (end_eval-start_eval)))},{str(datetime.timedelta(seconds = (end_eval-start_train)))}"
        runtime_sec = end_eval-start_train

        return_val = True

    except:
        auroc = -1
        f1=-1
        f1_macro = -1
        f1_micro = -1
        runtime_sec = end_train-start_train
        runtime = 'Failure'
        print("Failure!")

    results_file_metrics = f"{results_file}.csv"
    results_file_loss = f"{results_file}_loss.csv"

    #Append to results
    if os.path.exists(results_file_metrics):
        results = pd.read_csv(results_file_metrics)
    else:
        results = pd.DataFrame(columns=result_cols)

    result = pd.DataFrame(columns=result_cols,data=[[batch_name, disease,embedding,auroc,f1,f1_macro,f1_micro,runtime,runtime_sec,n_epoch, do, batch_size, hs, lr,str(cv)]])
    results = pd.concat([results,result])

    #Save results - overwrite so we can see progress
    results.to_csv(results_file_metrics, index=False)

    #write loss_history (Batch,Epoch,Loss)
    df_loss = pd.DataFrame(loss_history)
    df_loss = df_loss.rename(columns={0:"Loss"})
    df_loss['Epoch'] = df_loss.index + 1
    df_loss['Batch'] = batch_name
    df_loss['Disease'] = disease
    df_loss['Embedding'] = embedding
    df_loss['Epochs'] = n_epoch
    df_loss['Dropout'] = do
    df_loss['BatchSize'] = batch_size
    df_loss['Hidden'] = hs
    df_loss['LR'] = lr


    #Append to results
    if os.path.exists(results_file_loss):
        results = pd.read_csv(results_file_loss)
    else:
        results = pd.DataFrame(columns=result_loss_cols)

    results = pd.concat([results,df_loss])

    #Save results - overwrite so we can see progress
    results.to_csv(results_file_loss, index=False)

    return return_val

*** DL TF-IDF ***

In [370]:
for index, entry in enumerate(all_df['tok_lem_text']):
    Final_words = []
    #print(entry)
    for word in entry:
        #print(word)
        Final_words.append(word)
    all_df.loc[index, 'text_final'] = str(Final_words)



In [371]:
class TDFClinicalNotesDataset(Dataset):
    def __init__(self, X_array, y):
        df = pd.DataFrame(index=y.index)
        
        df['tfidf_vector'] = [vector.tolist() for vector in X_array]
        
        self.tfidf_vector = df.tfidf_vector.tolist()
        self.targets = y.tolist()

    def __getitem__(self, i):
        return (self.tfidf_vector[i], self.targets[i])
    
    def __len__(self):
        return len(self.targets)

def collate_fn(batch):
    tfidf = torch.tensor([item[0] for item in batch]).float()
    target = torch.tensor([int(item[1]==True) for item in batch]).long()

    return tfidf, target        

In [372]:
class ClincalNoteTDFNet(nn.Module):
    def __init__(self, tokens):
        super(ClincalNoteTDFNet, self).__init__()
        
        self.tokens = tokens

        self.hidden_dim1 = 128
        self.hidden_dim2 = 64
        self.hidden_dim3 = 32
        self.num_layers = 1

        #Because it is bidirectional, the output from LTSM is coming in twice the size of the hidden states required.
        #input is (batch, #of tokens * embedding_dimension)
        self.bilstm1 = nn.LSTM(input_size = self.tokens, hidden_size = self.hidden_dim1, bidirectional = True, batch_first = True, num_layers = self.num_layers, bias = False) 
        self.bilstm2 = nn.LSTM(input_size = self.hidden_dim1 * 2, hidden_size = self.hidden_dim2, bidirectional = True, batch_first = True, num_layers=self.num_layers, bias = False)
 
        self.fc1 = nn.Linear(self.hidden_dim2 * 2, self.hidden_dim2)
        self.fc2 = nn.Linear(self.hidden_dim2, 2)

 
    def forward(self, x):

        x, states = self.bilstm1(x)
        x, states = self.bilstm2(x)
        
        x = self.fc1(x)

        x = self.fc2(x)


        return x 


In [373]:
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
from sklearn.feature_selection import RFECV, RFE
from sklearn.tree import ExtraTreeClassifier
from sklearn.feature_selection import SelectKBest, SelectFromModel
from sklearn.feature_selection import f_classif, mutual_info_classif

def getVocab(X_train, y_train, feature, max_tokens):
 
    ## Step 1: Determine the Initial Vocabulary
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)
    vocab = list(tokenizer.word_index.keys())

    ## Step 2: Create term  matrix
    vectors = tokenizer.texts_to_matrix(X_train, mode='count')

    ## Do feature selection on term matrix (column headers are words)
    X = vectors
    y = y_train

    ##Choose algorithm
    if feature == 'SelectKBest':
        selector = SelectKBest(score_func=f_classif, k=max_tokens).fit(X,y)
    else: 
        if feature == 'InfoGainAttributeVal':
            #This should be similar to the InfoGain?
            selector = SelectKBest(score_func=mutual_info_classif, k=max_tokens).fit(X,y)
        else:
            #default to ExtraTreeClassifier
            estimator = ExtraTreeClassifier(random_state = seed)
            #selector = SelectFromModel(estimator, max_features = tokens,threshold=-np.inf)
            selector = SelectFromModel(estimator, max_features = max_tokens)
            selector = selector.fit(X, y)

    support_idx = selector.get_support(True)
    
    #print("Vocab:", [vocab[i-1].replace("'","") for i in support_idx])
    tokenizer2 = Tokenizer()
    tokenizer2.fit_on_texts([vocab[i-1].replace("'","") for i in support_idx])
    new_vocab = list(tokenizer2.word_index.keys())

    return new_vocab



In [374]:
def iterateTrainAndEvaluateTFIDF(df, k, disease_list, feature_list, lr_list, 
                            batch_name, results_file, batch_size, dataformat, device, tokens, epoch_list, cv = False, use_decay=False):

    for _,disease in enumerate(disease_list):
        for _,feature in enumerate(feature_list):
            for _,lr in enumerate(lr_list):
                for _, n_epoch in enumerate(epoch_list):
                    #Create a name for the model
                    model_name = f"{disease}_{feature}_{batch_name}"

                    disease_df = df[df['disease'] == disease].copy()

                    X_train, X_test, y_train, y_test = train_test_split(disease_df[dataformat], disease_df['judgment'], test_size=0.2, random_state=seed)

                    if feature != 'All':
                        vocab = getVocab(X_train,y_train, feature, tokens)
                        Tfidf_vect = TfidfVectorizer(max_features=tokens,vocabulary = vocab)
                    else:
                        Tfidf_vect = TfidfVectorizer(max_features=tokens)

                    X_train_values_list = Tfidf_vect.fit_transform(X_train).toarray()
                    X_training = pd.DataFrame(X_train_values_list, columns=Tfidf_vect.get_feature_names_out())
                    X_training = np.asarray(X_training, dtype=float)
                    X_training = torch.from_numpy(X_training).to(device)

                    X_test_values_list = Tfidf_vect.transform(X_test).toarray()
                    X_testing = pd.DataFrame(X_test_values_list, columns=Tfidf_vect.get_feature_names_out())
                    X_testing = np.asarray(X_testing, dtype=float)
                    X_testing = torch.from_numpy(X_testing).to(device)

                    tokens_to_use = X_training.shape[1]
                    print(tokens_to_use)

                    #Create model
                    model = ClincalNoteTDFNet(tokens = tokens_to_use)
                    model = model.to(device)

                    ds_train = TDFClinicalNotesDataset(X_training, y_train)
                    ds_test = TDFClinicalNotesDataset(X_testing, y_test)

                    #Load Data 
                    train_loader = torch.utils.data.DataLoader(ds_train, batch_size = batch_size, collate_fn=collate_fn)
                    val_loader = torch.utils.data.DataLoader(ds_test, batch_size = batch_size,collate_fn=collate_fn)

                    model_desc = f"{disease}_{feature}"

                    trainAndEvaluate(train_loader, val_loader, model, model_desc, batch_name, results_file, disease, lr, dataformat, feature, device, n_epoch, False, use_decay)

                    #Save model
                    torch.save(model.state_dict(), f'{MODELS_PATH}{model_name}.pkl')

                    #Delete model
                    del model

                

In [375]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(f'Using device: {device}')

#Override these if need be
disease_list = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'OSA', 'PVD', 'Venous Insufficiency', 'Obesity']
#disease_list = ['Asthma']
feature_list = ['All','ExtraTreeClassifier','SelectKBest','InfoGainAttributeVal']


#0.01 seems to be the most effective with no decay
lr_list = [0.1,0.01,0.001]
#lr_list = [0.01]
epoch_list = [10,25,50,100]

#training parameters
n_epoch = 50
batch_size = 128
k = 2

#These should not change
dataformat = 'text_final'
tokens = 600

results_file = f'{RESULTS_PATH}DL_tfidf_results'
result_time = datetime.datetime.now()
result_name = result_time.strftime("%Y-%m-%d-%H-%M-%S")
batch_name = f'DL_tfidf_results_{result_name}'

#commented out because working on Embeddings
#iterateTrainAndEvaluateTFIDF(all_df, k, disease_list, feature_list, lr_list, batch_name, results_file, batch_size, dataformat, device, tokens, epoch_list, False, False)


Using device: cpu


****DL Model using word embeddings****

First we start by creating a dataset.  Note this will have to take the disease as part of the init and filter just for those records.

In [376]:

class ClinicalNoteDataset(Dataset):

    def __init__(self, dataframe, disease, dataformat):
        """
        TODO: init the Dataset instance.  datafomat is just the column to use from the dataframe 'vector_tokenized' , 'one_hot'
        """
        # your code here
        self.disease = disease
        self.dataformat = dataformat
        self.df = dataframe[dataframe['disease'] == disease].copy()
        self.df = self.df.reset_index()

    def __len__(self):
        """
        TODO: Denotes the total number of samples
        """
        return len(self.df)

    def __getitem__(self, i):
        """
        TODO: Generates one sample of data
            return X, y for the i-th data.
        """
        #Cannot make tensors yet, will need to happen in collate
        Y = self.df.iloc[i]['judgment']
        X = self.df.iloc[i][self.dataformat]

        return X,Y
        
def vectorize_batch_GloVe(batch):
    embedding_size_used = 300
    vec = torchtext.vocab.GloVe(name='6B', dim=embedding_size_used)    
    Xi, Yi = batch[0]
    batch_size = len(batch)

    X = torch.zeros(batch_size, len(Xi), embedding_size_used, dtype=torch.float)
    Y = torch.zeros((batch_size), dtype=torch.long)
    
    for i in range(len(batch)):
        x, y = batch[i]
        #vectors = vec.get_vecs_by_tokens(voc.lookup_tokens(x.tolist()))
        vectors = vec.get_vecs_by_tokens(x)

        X[i] = vectors.float()
        Y[i] = torch.tensor(float(y == True))

    return X,Y

def vectorize_batch_FastText(batch):
    embedding_size_used = 300
    vec = torchtext.vocab.FastText()
    Xi, Yi = batch[0]
    batch_size = len(batch)

    X = torch.zeros(batch_size, len(Xi), embedding_size_used, dtype=torch.float)
    Y = torch.zeros((batch_size), dtype=torch.long)

    for i in range(len(batch)):
        x, y = batch[i]
        #vectors = vec.get_vecs_by_tokens(voc.lookup_tokens(x.tolist()))
        vectors = vec.get_vecs_by_tokens(x)

        X[i] = vectors.float()
        Y[i] = torch.tensor(float(y == True))

    return X,Y 
 
def vectorize_batch_USE(batch):
    embedding_size_used = 512

    Xi, Yi = batch[0]
    batch_size = len(batch)

    X = torch.zeros(batch_size, len(Xi), embedding_size_used, dtype=torch.float)
    Y = torch.zeros((batch_size), dtype=torch.long)

    for i in range(len(batch)):
        x, y = batch[i]
        
        tensor_flow_vectors = embed(x)
        array_vectors = tensor_flow_vectors.numpy()

        X[i] = torch.tensor(array_vectors).float()
        Y[i] = torch.tensor(float(y == True))

    return X,Y 
          


In [377]:
##Test DataLoader
#batch_size = 128
#train_loader = torch.utils.data.DataLoader(ClinicalNoteDataset(train_df, 'Asthma', 'vector_tokenized'), batch_size = batch_size, shuffle=True, collate_fn=vectorize_batch_FastText)
#val_loader = torch.utils.data.DataLoader(ClinicalNoteDataset(test_df, 'Asthma', 'vector_tokenized'), batch_size = batch_size, shuffle=False, collate_fn=vectorize_batch_FastText)

#print("# of train batches:", len(train_loader))
#print("# of val batches:", len(val_loader))

#train_iter = iter(train_loader)
#x,y = next(train_iter)

#print(x.shape)
#print(y.shape)


In [378]:
class ClincalNoteEmbeddingNet(nn.Module):
    def __init__(self, embedding_type, max_tokens, dropout, hidden_size):
        super(ClincalNoteEmbeddingNet, self).__init__()
        
        self.max_tokens = max_tokens
        self.dropout = dropout
        self.hidden_size = hidden_size

        if(embedding_type == 'USE'):
            self.embedding_dimension = 512
        else:
            self.embedding_dimension = 300

        self.hidden_dim1 = self.hidden_size
        self.hidden_dim2 = int(self.hidden_size/2)
        if self.dropout > 0:
            self.num_layers = 2
        else:
            self.num_layers = 1


        #Because it is bidirectional, the output from LTSM is coming in twice the size of the hidden states required.
        #input is (batch, #of tokens * embedding_dimension)
        self.bilstm1 = nn.LSTM(input_size = self.embedding_dimension, hidden_size = int(self.hidden_dim1/2), bidirectional = True, dropout = self.dropout, 
                               batch_first = True, num_layers = self.num_layers) 
        
        self.bilstm2 = nn.LSTM(input_size = self.hidden_dim1, hidden_size = int(self.hidden_dim2/2), bidirectional = True, dropout = self.dropout, 
                               batch_first = True, num_layers=self.num_layers)

        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(self.hidden_dim2 * self.max_tokens, 2)


    def forward(self, x):

        x, states = self.bilstm1(x)
        x, states = self.bilstm2(x)
        
        x = self.flatten(x)
        x = self.fc1(x)

        return x 



Need to create a loop to train and evaluate

In [379]:
def iterateTrainAndEvaluateHP(df, k, disease_list, embedding_list, lr_list, 
                            batch_name, results_file, device, epoch_list, dropout_list, batch_size_list, hs_list, use_decay = False):

    for _,disease in enumerate(disease_list):
        for _,embedding in enumerate(embedding_list):
            for _,lr in enumerate(lr_list):
                for _, n_epoch in enumerate(epoch_list):
                    for _,batch_size in enumerate(batch_size_list):
                        for _, do in enumerate(dropout_list):
                            for _, hs in enumerate(hs_list):
                                #Create a name for the model
                                model_name = f"{disease}_{embedding}_{batch_name}"

                                #Create model
                                if embedding == 'USE':
                                    model_tokens = max_sentences
                                else:
                                    model_tokens = max_tokens
                                    
                                model = ClincalNoteEmbeddingNet(embedding, max_tokens = model_tokens, dropout = do, hidden_size = hs)
                                model = model.to(device)

                                if embedding == 'GloVe':
                                    custom_collate=vectorize_batch_GloVe
                                    dataformat = 'vector_tokenized'
                                if embedding == 'FastText':
                                    custom_collate=vectorize_batch_FastText
                                    dataformat = 'vector_tokenized'
                                if embedding == 'USE':
                                    custom_collate=vectorize_batch_USE
                                    dataformat = 'sentence_tokenized'

                                ds = ClinicalNoteDataset(df, disease, dataformat)
                                ds_train, ds_test = train_test_split(ds, test_size=0.20, shuffle=True, random_state = seed)

                                #Load Data 
                                train_loader = torch.utils.data.DataLoader(ds_train, batch_size = batch_size, collate_fn=custom_collate)
                                val_loader = torch.utils.data.DataLoader(ds_test, batch_size = batch_size, collate_fn=custom_collate)
                                
                                model_desc = f"{disease}_{embedding}"

                                trainAndEvaluate(train_loader, val_loader, model, model_desc, batch_name, results_file, disease, lr, dataformat, embedding, device, n_epoch,  do, batch_size, hs, False, use_decay)

                                #Save model - don't need to save for hyper parameter tuning
                                #torch.save(model.state_dict(), f'{MODELS_PATH}{model_name}.pkl')

                                #Delete model
                                del model



In [380]:
def iterateTrainAndEvaluate(df, k, disease_list, embedding_list, lr_list, 
                            batch_name, results_file, device, epoch_list, do, batch_size, hs, cv = False, use_decay = False):

    for _,disease in enumerate(disease_list):
        for embedding_idx,embedding in enumerate(embedding_list):
            lr = lr_list[embedding_idx]
            n_epoch = epoch_list[embedding_idx]
            #Create a name for the model
            model_name = f"{disease}_{embedding}_{batch_name}"

            #Create model
            if embedding == 'USE':
                model_tokens = max_sentences
            else:
                model_tokens = max_tokens
                
            model = ClincalNoteEmbeddingNet(embedding, max_tokens = model_tokens, dropout = do, hidden_size=hs)
            model = model.to(device)

            if embedding == 'GloVe':
                custom_collate=vectorize_batch_GloVe
                dataformat = 'vector_tokenized'
            if embedding == 'FastText':
                custom_collate=vectorize_batch_FastText
                dataformat = 'vector_tokenized'
            if embedding == 'USE':
                custom_collate=vectorize_batch_USE
                dataformat = 'sentence_tokenized'

            ds = ClinicalNoteDataset(df, disease, dataformat)
            ds_train, ds_test = train_test_split(ds, test_size=0.20, shuffle=True, random_state = seed)

            #Load Data 
            train_loader = torch.utils.data.DataLoader(ds_train, batch_size = batch_size, collate_fn=custom_collate)
            val_loader = torch.utils.data.DataLoader(ds_test, batch_size = batch_size, collate_fn=custom_collate)
            
            model_desc = f"{disease}_{embedding}"

            trainAndEvaluate(train_loader, val_loader, model, model_desc, batch_name, results_file, disease, lr, dataformat, embedding, device, n_epoch, do, batch_size, hs, False, use_decay)

            #Save model
            torch.save(model.state_dict(), f'{MODELS_PATH}{model_name}.pkl')

            #Delete model
            del model

            if cv:
                #note, cross validation is only used to validate the model works consistently
                splits=KFold(n_splits=k,shuffle=True,random_state=seed)

                for fold, (train_idx,val_idx) in enumerate(splits.split(np.arange(len(ds)))):
                    #for now, let's keep the results at the fold level
                    model = ClincalNoteEmbeddingNet(embedding, max_tokens = max_tokens)
                    model = model.to(device)
                    
                    train_sampler = SubsetRandomSampler(train_idx)
                    val_sampler = SubsetRandomSampler(val_idx)
                    #Load Data 
                    train_loader = torch.utils.data.DataLoader(ds, batch_size = batch_size, sampler=train_sampler, collate_fn=custom_collate)
                    val_loader = torch.utils.data.DataLoader(ds, batch_size = batch_size, sampler=val_sampler, collate_fn=custom_collate)
                    
                    model_desc = f"{disease}_{embedding}_Fold{fold+1}"

                    trainAndEvaluate(train_loader, val_loader, model, model_desc, batch_name, results_file, disease, lr, dataformat, embedding, device, n_epoch, do, batch_size, hs, cv, use_decay)

                    del model

In [381]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(f'Using device: {device}')

#Override these if need be
#disease_list = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'OSA', 'PVD', 'Venous Insufficiency', 'Obesity']
disease_list = ['Asthma']
embedding_list = ['GloVe','FastText','USE']
#embedding_list = ['GloVe']
epoch_list = [25,50,100]
#0.01 seems to be the most effective, although FastText prefers 0.001
lr_list = [0.01,0.001]
dropout_list = [0, 0.1, 0.5]
batch_size_list = [64,128] #can't go above 128, 256 worked, but pushed memoery to limit
hs_list = [64, 128]


results_file = f'{RESULTS_PATH}DL_embedding_results'

#training parameters
k = 2

#These should not change

result_time = datetime.datetime.now()
result_name = result_time.strftime("%Y-%m-%d-%H-%M-%S")
batch_name = f'DL_embedding_results_{result_name}'

iterateTrainAndEvaluateHP(all_df_expanded, k, disease_list, embedding_list, lr_list, batch_name, results_file, device, epoch_list, dropout_list, batch_size_list, hs_list, False)



Using device: cpu


Training Asthma_GloVe-Lr0.01-Epoch1of25-BatchSize64-HiddenState64-Dropout0...:  80%|████████  | 12/15 [00:40<00:09,  3.12s/it]

In [ ]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
print(f'Using device: {device}')

#Override these if need be
#disease_list = ['Asthma', 'CAD', 'CHF', 'Depression', 'Diabetes', 'Gallstones', 'GERD', 'Gout', 'Hypercholesterolemia', 'Hypertension', 'Hypertriglyceridemia', 'OA', 'OSA', 'PVD', 'Venous Insufficiency', 'Obesity']
disease_list = ['Asthma']
embedding_list = ['GloVe','FastText','USE']
#embedding_list = ['GloVe']
#epoch_list = [25,25,50]
epoch_list = [1,1,1]
lr_list = [0.01,0.001,0.01]
dropout = .1
hs = 64


results_file = f'{RESULTS_PATH}DL_embedding_results'

#training parameters
batch_size = 256
k = 2

#These should not change

result_time = datetime.datetime.now()
result_name = result_time.strftime("%Y-%m-%d-%H-%M-%S")
batch_name = f'DL_embedding_results_{result_name}'
#iterateTrainAndEvaluate(all_df_expanded, k, disease_list, embedding_list, lr_list, batch_name, results_file, device, epoch_list, dropout, batch_size, hs, False, False)